In [4]:
#importing necessary modules
import cv2
import os
import time
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
#capturing images and labelling them
signs = ['thumbsup','thumbsdown','hello','thankyou','peace','namaste','iloveyou','ok']
number_of_images = 15


images_loc_train = os.path.join('Project','workspace','images','train')
images_loc_test = os.path.join('Project','workspace','images','test')    
for sign in signs:
    capture = cv2.VideoCapture(0)
    print('capturing images of {}'.format(sign))
    for capture_number in range(number_of_images):
        print('collecting image number {}'.format(capture_number))
        time.sleep(5)
        ret, frame = capture.read()
        if capture_number<12:
            image_name = os.path.join(images_loc_train,sign+'_'+str(capture_number)+'.jpg')
        else:
            image_name = os.path.join(images_loc_test,sign+'_'+str(capture_number)+'.jpg')
        cv2.imwrite(image_name, frame)
        cv2.imshow('frame', frame)
        print('done, ready the next image')
        
        if cv2.waitKey(1) & 0xff == ord('q'):
            capture.release()
            break
            
capture.release()
cv2.destroyAllWindows()

LabelIMG_loc = os.path.join('Tensorflow','labelimg')
!cd {LabelIMG_loc} && python labelImg.py



In [2]:
#training
labels = [{'name':'thumbsup', 'id':1}, {'name':'thumbsdown', 'id':2}, {'name':'hello', 'id':3}, {'name':'thankyou', 'id':4},
          {'name':'peace','id':5},{'name':'namaste','id':6},{'name':'iloveyou','id':7},{'name':'ok','id':8}]

label_map_loc = os.path.join('Project','workspace','annotations','label_map.pbtxt')
with open(label_map_loc, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

images_loc_train = os.path.join('Project','workspace','images','train')
images_loc_test = os.path.join('Project','workspace','images','test')
train_record_loc = os.path.join('Project','workspace','annotations', 'train.record')
test_record_loc = os.path.join('Project','workspace','annotations','test.record')
pretrained_model_loc = os.path.join('Tensorflow','models','research', 'object_detection', 'model_main_tf2.py')
Project_Model_Loc = os.path.join('project', 'workspace','Project_Model')
Pipeline_config_loc = os.path.join('Project', 'workspace','Project_Model', 'pipeline.config')

!python Tensorflow/scripts/generate_tfrecord.py -x {images_loc_train} -l {label_map_loc} -o {train_record_loc}
!python Tensorflow/scripts/generate_tfrecord.py -x {images_loc_test} -l {label_map_loc} -o {test_record_loc}

TRAINING_SCRIPT = os.path.join('Tensorflow','models' ,'research', 'object_detection', 'model_main_tf2.py')
Checkpoint_path = os.path.join('Project','workspace','Project_Model')
pipeline_path = os.path.join('Project','workspace','Project_Model','pipeline.config')

command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=3000".format(TRAINING_SCRIPT, Checkpoint_path,pipeline_path)
!{command}

Project_Model_Loc = os.path.join('Project','workspace','Project_Model')
Pipeline_config_loc = os.path.join('Project', 'workspace','Project_Model', 'pipeline.config')
label_map_loc = os.path.join('Project','workspace','annotations','label_map.pbtxt')



Successfully created the TFRecord file: Project\workspace\annotations\train.record
Successfully created the TFRecord file: Project\workspace\annotations\test.record


In [5]:
Project_Model_Loc = os.path.join('project', 'workspace','Project_Model')
Pipeline_config_loc = os.path.join('Project', 'workspace','Project_Model', 'pipeline.config')

checkpoint_loc = os.path.join(Project_Model_Loc,'ckpt-6')
configs = config_util.get_configs_from_pipeline_file(Pipeline_config_loc)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(checkpoint_loc).expect_partial()

def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [6]:
#live object detection
label_map_loc = os.path.join('Project','workspace','annotations','label_map.pbtxt')

live_feed = cv2.VideoCapture(0)
width = int(live_feed.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(live_feed.get(cv2.CAP_PROP_FRAME_HEIGHT))
category_index = label_map_util.create_category_index_from_labelmap(label_map_loc)

while live_feed.isOpened(): 
    ret, frame = live_feed.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        live_feed.release()
        cv2.destroyAllWindows()
        break

In [7]:
live_feed.release()
cv2.destroyAllWindows()